<a href="https://colab.research.google.com/github/sunnyinAI/Diffusion_DisparitySum/blob/main/submodlib_and_split_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from submodlib import FacilityLocationFunction, DisparitySumFunction, DisparityMinFunction, LogDeterminantFunction, FacilityLocationMutualInformationFunction
import os
import shutil
import numpy as np
import time

In [ ]:
groundData = pd.read_excel('pneumoniaMNIST_feat_lake.xlsx')

In [ ]:
# repData_transpose = repData.transpose()

In [ ]:
groundData_transpose = groundData.transpose()
num_images = len(groundData_transpose)
num_features = 512
budget = 256

algo = 'DisparitySumFunction'
# algo = 'FacilityLocationMutualInformationFunction'
#from submodlib import LogDeterminantMutualInformationFunction
row = 0
index = 1

supported_algos = ['FacilityLocationFunction', 'DisparitySumFunction', 'DisparityMinFunction', 'LogDeterminantFunction']

In [ ]:
len(groundData_transpose)

2562

In [ ]:
# grounds = [list(groundData_transpose[x]) for x in groundData_transpose.columns]
# len(grounds)

In [ ]:
start_time = time.time()
print('Starting selection algo ...')
if algo == 'FacilityLocationFunction':
    # objFL = FacilityLocationFunction(n=len(grounds), data=groundData, separate_rep=True, n_rep=36, data_rep=repData, mode="dense", metric="euclidean")
    objFL = FacilityLocationFunction(n=num_images, data=np.array(groundData_transpose), separate_rep=False, mode="dense", metric="euclidean")
    greedyList = objFL.maximize(budget=budget ,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
elif algo == 'DisparitySumFunction':
    objFL = DisparitySumFunction(n=num_images, data=np.array(groundData_transpose), mode="dense", metric="euclidean")
    greedyList = objFL.maximize(budget=budget ,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
elif algo == 'DisparityMinFunction':
    objFL = DisparityMinFunction(n=num_images, data=np.array(groundData_transpose), mode="dense", metric="euclidean")
    greedyList = objFL.maximize(budget=budget ,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
elif algo == 'LogDeterminantFunction':
    lambda_value = 1
    objFL = LogDeterminantFunction(n=num_images, data=np.array(groundData_transpose), mode="dense", metric="euclidean", lambdaVal=lambda_value)
    greedyList = objFL.maximize(budget=budget ,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
# elif algo == 'FacilityLocationMutualInformationFunction':
#     obj = FacilityLocationMutualInformationFunction(n=num_images, num_queries=len(repData_transpose), data=np.array(groundData_transpose),queryData=np.array(repData_transpose), metric="euclidean", magnificationEta=etas)
#     greedyList = obj.maximize(budget=budget,optimizer='NaiveGreedy', stopIfZeroGain=False,
#                               stopIfNegativeGain=False, verbose=False)

end_time = time.time()

print(f'Time taken - {np.round((end_time - start_time) / 60, 2)}')

# greedys = [[grounds[i][x[0]] for x in greedyList] for i in range(num_features)]

Starting selection algo ...
Time taken - 0.15


[||||||||||||||||    ]84% [Iteration 213 of 256]|||                ]23% [Iteration 59 of 256]

In [ ]:
greedyList

[||||||||||||||||||||]100% [Iteration 256 of 256]

[(2561, 0.0),
 (1484, 0.046526193618774414),
 (2174, 0.09092110395431519),
 (1635, 0.130021333694458),
 (441, 0.17088276147842407),
 (1588, 0.21613019704818726),
 (1319, 0.2583642601966858),
 (1425, 0.30116504430770874),
 (1287, 0.3480195999145508),
 (1107, 0.38590341806411743),
 (1026, 0.42272597551345825),
 (2000, 0.46693289279937744),
 (2178, 0.5097097754478455),
 (981, 0.5472741723060608),
 (113, 0.5923492908477783),
 (1074, 0.6307178139686584),
 (1525, 0.6701918244361877),
 (1837, 0.70767742395401),
 (2051, 0.7531453967094421),
 (127, 0.7902838587760925),
 (919, 0.828252375125885),
 (2491, 0.8680713176727295),
 (1414, 0.9134705066680908),
 (2324, 0.9529381394386292),
 (1951, 0.9924570918083191),
 (799, 1.0323436856269836),
 (145, 1.0710160732269287),
 (44, 1.1153711080551147),
 (503, 1.1504766345024109),
 (2120, 1.1902498006820679),
 (1858, 1.2257946133613586),
 (710, 1.268502950668335),
 (2303, 1.309932827949524),
 (963, 1.346666395664215),
 (1812, 1.3830522894859314),
 (78, 1.42

In [ ]:
selected_image_idxs = [greedyList[i][0] for i in range(len(greedyList))]

In [ ]:
selected_image_names = [groundData_transpose.index[i] for i in selected_image_idxs]

In [ ]:
# Splitting data for creating train_baseline
try:
    os.mkdir('train_w_submodlib')
except FileExistsError:
    pass

try:
    os.mkdir('train_w_submodlib/train_0')
except FileExistsError:
    pass

try:
    os.mkdir('train_w_submodlib/train_1')
except FileExistsError:
    pass

In [ ]:
for i in range(len(selected_image_names)):

    # print(selected_image_names[i])
    img_file_source = os.path.join('lake', selected_image_names[i])
    if selected_image_names[i].split('.')[0][0] == '0':
        img_file_destination = os.path.join('train_w_submodlib', 'train_0', selected_image_names[i])
    elif selected_image_names[i].split('.')[0][0] == '1':
        img_file_destination = os.path.join('train_w_submodlib', 'train_1', selected_image_names[i])
    else:
        print('Image file destination not defined')
        img_file_destination = None

    print(img_file_source)
    print(img_file_destination)
    shutil.copy(img_file_source, img_file_destination)

lake/0_image_325.jpg
train_w_submodlib/train_0/0_image_325.jpg
lake/1_image_963.jpg
train_w_submodlib/train_1/1_image_963.jpg
lake/0_image_395.jpg
train_w_submodlib/train_0/0_image_395.jpg
lake/1_image_1248.jpg
train_w_submodlib/train_1/1_image_1248.jpg
lake/1_image_1182.jpg
train_w_submodlib/train_1/1_image_1182.jpg
lake/1_image_349.jpg
train_w_submodlib/train_1/1_image_349.jpg
lake/0_image_1691.jpg
train_w_submodlib/train_0/0_image_1691.jpg
lake/0_image_1089.jpg
train_w_submodlib/train_0/0_image_1089.jpg
lake/1_image_145.jpg
train_w_submodlib/train_1/1_image_145.jpg
lake/1_image_836.jpg
train_w_submodlib/train_1/1_image_836.jpg
lake/1_image_1659.jpg
train_w_submodlib/train_1/1_image_1659.jpg
lake/0_image_608.jpg
train_w_submodlib/train_0/0_image_608.jpg
lake/0_image_149.jpg
train_w_submodlib/train_0/0_image_149.jpg
lake/1_image_1733.jpg
train_w_submodlib/train_1/1_image_1733.jpg
lake/1_image_211.jpg
train_w_submodlib/train_1/1_image_211.jpg
lake/1_image_806.jpg
train_w_submodlib/trai

In [ ]:
# Creating split for training WGAN for normal and pneumonia

# Splitting data for creating train_baseline

try:
    os.mkdir('diff_train')
except FileExistsError:
    pass

try:
    os.mkdir('diff_train/train_0')
except FileExistsError:
    pass

try:
    os.mkdir('diff_train/train_1')
except FileExistsError:
    pass

try:
    os.mkdir('diff_train/train_0/train_0')
except FileExistsError:
    pass

try:
    os.mkdir('diff_train/train_1/train_1')
except FileExistsError:
    pass

In [ ]:
for i in range(len(selected_image_names)):

    # print(selected_image_names[i])

    img_file_source = os.path.join('lake', selected_image_names[i])

    if selected_image_names[i].split('.')[0][0] == '0':
        img_file_destination = os.path.join('diff_train', 'train_0', 'train_0', selected_image_names[i])
    elif selected_image_names[i].split('.')[0][0] == '1':
        img_file_destination = os.path.join('diff_train', 'train_1', 'train_1', selected_image_names[i])
    else:
        print('Image file destination not defined')
        img_file_destination = None

    # print(img_file_source)
    # print(img_file_destination)

    shutil.copy(img_file_source, img_file_destination)